In [1]:
import xarray as xr 
import numpy as np
from netCDF4 import Dataset
from scipy.stats import rankdata

This file computes CPA plots for HRES and persistency forecast in comparison with reanalysis for the variables: 
- 2m Temperature
- Wind speed
- Precipitation 

Data is downloaded from either TIGGE (HRES) or ERA5 (Reanalysis for single level) at initialtime 00:00 UTC for the years 2007-2018. Note that for precipitation   summation over hourly rainfall from ERA4 is necessary. Both wind speed (sqrt(u^2+v^2)) and 2m Temperature are instantaneous. From global dataset select box 335E,44.5E,25N,74.5N.  

In [ ]:
# computation of CPA plot with reanalysis for 2m Temperature
# time: 00:00
# years: 2014 - 2018 and for persistence forecast 27.12.2003 - 31.12.2003

target_obs = '/home/eva-maria/lsdf_data/2mTemp_Reanalysis/Boxreanalysis_2003-2018.nc'
DS_observation = xr.open_dataset(target_obs)

m = 0
lead = 1
k = 5
CPA_All = np.zeros(57)
CPAEurop = np.zeros((199,279))
indices_start0 = np.array([0,91,182,274,366,456,547,639, 731, 821, 912, 1004])
indices_end0 = np.array([366,456,547, 639, 731, 821, 912, 1004, 1096, 1186, 1277, 1369])
indices_start = np.array([0,90,181,273,365,456,547,639,731,821,912,1004,1096,1186,1277,1369,1461,1551,1642,1734,1826,1917,2008,2100,2192,2282,2373,2465,2557,2647,2738,2830,2922,3012,3103,3195,3287,3378,3469,3561,3653,3743,3834,3926,4018])
indices_end = np.array([274,365,  456,  548,  640,  730,  821,  913, 1005,1095, 1186, 1278, 1370, 1460, 1551, 1643, 1735, 1826, 1917, 2009,2101, 2191, 2282, 2374, 2466, 2556, 2647, 2739, 2831, 2921, 3012,3104, 3196, 3287, 3378, 3470, 3562, 3652, 3743, 3835, 3927, 4017,4108,4200,4292])
indices_start0 =  indices_start0 + k  
indices_end0 = indices_end0 +k
indices_start = indices_start + 1096 + k
indices_end = indices_end + 1096+91 + k
ind_start = np.concatenate((indices_start0, indices_start), axis=None)
ind_end = np.concatenate((indices_end0, indices_end), axis=None)
for indx in range(0,57):
    start = ind_start[indx]
    end = ind_end[indx]

    predictor_nc = np.asarray(DS_observation.t2m[(start-lead):(end-lead),:,:])
    response_nc = np.asarray(DS_observation.t2m[start:end,:,:]) 
    
    response = response_nc.flatten()
    predictor = predictor_nc.flatten()
             
    resOrder = np.argsort(response)
    resSort = response[resOrder] 
    preSort = predictor[resOrder]                
    preRank = rankdata(preSort, method='average')
    resRank = rankdata(resSort, method='average')
    resClass = rankdata(resSort, method='dense')
    CPA_All[m] = (np.cov(resClass,preRank)[0][1]/np.cov(resClass,resRank)[0][1]+1)/2 
    print(CPA_All[m])
    m = m+1
print(CPA_All)

np.savetxt('CPATemperature_24.txt',CPA_All)

In [ ]:
# computation of CPA plot with tigge HRES for 2m temperature
# read in Forecast24, Forecast48, Forecast72, Forecast96, Forecast120 depending on lead time
# k = 5 fix since reanalysis considered as observation (response)

target_obs = '/home/eva-maria/lsdf_data/2mTemp_Reanalysis/Boxreanalysis_2003-2018.nc'
DS_observation = xr.open_dataset(target_obs)
target_hres = '/home/eva-maria/lsdf_data/Lead24/Forecast24.nc'
DS_predictor = xr.open_dataset(target_hres)

m = 0
k = 5
CPA_All = np.zeros(45)
CPAEurop = np.zeros((199,279))
# indices for reanalysis
indices_start = np.array([0,90,181,273,365,456,547,639,731,821,912,1004,1096,1186,1277,1369,1461,1551,1642,1734,1826,1917,2008,2100,2192,2282,2373,2465,2557,2647,2738,2830,2922,3012,3103,3195,3287,3378,3469,3561,3653,3743,3834,3926,4018])
indices_end = np.array([274,365,  456,  548,  640,  730,  821,  913, 1005,1095, 1186, 1278, 1370, 1460, 1551, 1643, 1735, 1826, 1917, 2009,2101, 2191, 2282, 2374, 2466, 2556, 2647, 2739, 2831, 2921, 3012,3104, 3196, 3287, 3378, 3470, 3562, 3652, 3743, 3835, 3927, 4017,4108,4200,4292])
indices_start = indices_start + 1096 + k
indices_end = indices_end + 1096+91 + k

# indies for HRES forecast
indices_start_h = np.array([0,90,181,273,365,456,547,639,731,821,912,1004,1096,1186,1277,1369,1461,1551,1642,1734,1826,1917,2008,2100,2192,2282,2373,2465,2557,2647,2738,2830,2922,3012,3103,3195,3287,3378,3469,3561,3653,3743,3834,3926,4018])
indices_end_h = np.array([365,  456,  547,  639,  731,821,912,1004,1096,1186,1277,1369,1461,1551,1642,1734,1826, 1917, 2008,2100, 2192, 2282, 2373, 2465, 2557, 2647, 2738, 2830, 2922, 3012,3103,3195,3287,3378,3469,3561,3653,3743,3834,3926,4018,4108,4199,4291,4383])



for indx in range(0,45):
    start = indices_start[indx]
    end = indices_end[indx]
    start_h =  indices_start_h[indx]
    end_h = indices_end_h[indx]
    predictor_nc = np.asarray(DS_predictor.t2m[start_h:end_h,:,:])
    response_nc = np.asarray(DS_observation.t2m[start:end,:,:])

    response = response_nc.flatten()
    predictor = predictor_nc.flatten()
             
    resOrder = np.argsort(response)
    resSort = response[resOrder] 
    preSort = predictor[resOrder]                
    preRank = rankdata(preSort, method='average')
    resRank = rankdata(resSort, method='average')
    resClass = rankdata(resSort, method='dense')
    CPA_All[m] = (np.cov(resClass,preRank)[0][1]/np.cov(resClass,resRank)[0][1]+1)/2 
    print(CPA_All[m])
    m = m+1
print(CPA_All)

np.savetxt('CPATemperature_hres_24.txt',CPA_All)

In [ ]:
# computation of CPA plot with reanalysis for U and V Wind only for 2007 until 2018
# data is available from 2014 - 2018. Therefore +1096. Since no day from 2003 necessary in CPA computation 
# using persistence for the years 2007 - 2018 use k=0

target_obs1 = '/home/eva-maria/lsdf_data/Wind/Reanalysis/UWind_reanalysis_all.nc'
DS_observation_u = xr.open_dataset(target_obs1)

target_obs2 = '/home/eva-maria/lsdf_data/Wind/Reanalysis/VWind_reanalysis_all.nc'
DS_observation_v = xr.open_dataset(target_obs2)


m = 0
lead = 1
k = 0
CPA_All = np.zeros(45)
CPAEurop = np.zeros((199,279))
#indices_start0 = np.array([0,91,182,274,366,456,547,639, 731, 821, 912, 1004])
#indices_end0 = np.array([366,456,547, 639, 731, 821, 912, 1004, 1096, 1186, 1277, 1369])
indices_start = np.array([0,90,181,273,365,456,547,639,731,821,912,1004,1096,1186,1277,1369,1461,1551,1642,1734,1826,1917,2008,2100,2192,2282,2373,2465,2557,2647,2738,2830,2922,3012,3103,3195,3287,3378,3469,3561,3653,3743,3834,3926,4018])
indices_end = np.array([274,365,  456,  548,  640,  730,  821,  913, 1005,1095, 1186, 1278, 1370, 1460, 1551, 1643, 1735, 1826, 1917, 2009,2101, 2191, 2282, 2374, 2466, 2556, 2647, 2739, 2831, 2921, 3012,3104, 3196, 3287, 3378, 3470, 3562, 3652, 3743, 3835, 3927, 4017,4108,4200,4292])
#indices_start0 =  indices_start0 + k  
#indices_end0 = indices_end0 +k
ind_start = indices_start + 1096 + k
ind_end = indices_end + 1096+91 + k
#ind_start = np.concatenate((indices_start0, indices_start), axis=None)
#ind_end = np.concatenate((indices_end0, indices_end), axis=None)

for indx in range(0,45):
    start = ind_start[indx]
    end = ind_end[indx]
    predictor_nc_u = np.asarray(DS_observation_u.u10[(start-lead):(end-lead),:,:])
    predictor_nc_v = np.asarray(DS_observation_v.v10[(start-lead):(end-lead),:,:])
    response_nc_u = np.asarray(DS_observation_u.u10[start:end,:,:]) 
    response_nc_v = np.asarray(DS_observation_v.v10[start:end,:,:]) 
    
    response_u = response_nc_u.flatten()
    response_v = response_nc_v.flatten()
    predictor_u = predictor_nc_u.flatten()
    predictor_v = predictor_nc_v.flatten()
    
    response = np.sqrt(np.square(response_u)+np.square(response_v))
    predictor = np.sqrt(np.square(predictor_u)+np.square(predictor_v))
             
    resOrder = np.argsort(response)
    resSort = response[resOrder] 
    preSort = predictor[resOrder]                
    preRank = rankdata(preSort, method='average')
    resRank = rankdata(resSort, method='average')
    resClass = rankdata(resSort, method='dense')
    CPA_All[m] = (np.cov(resClass,preRank)[0][1]/np.cov(resClass,resRank)[0][1]+1)/2 
    print(CPA_All[m])
    m = m+1
print(CPA_All)

np.savetxt('CPAWindSpeed_24.txt',CPA_All)

In [ ]:
# compute HRES CPA plot for U and V wind
target_fore = '/home/eva-maria/lsdf_data/Wind/Lead72/WForecast_72.nc' 
target_obs =  '/home/eva-maria/lsdf_data/Wind/UWboxreanalysis.nc'
target_obs2 = '/home/eva-maria/lsdf_data/Wind/VWboxreanalysis.nc'
DS_forecast = xr.open_dataset(target_fore)
DS_observation = xr.open_dataset(target_obs)
DS_observation2 = xr.open_dataset(target_obs2)



m = 0
CPA_All = np.zeros(45)
indices_start = [0,90,181,273,365,456,547,639,731,821,912,1004,1096,1186,1277,1369,1461,1551,1642,1734,1826,1917,2008,2100,2192,2282,2373,2465,2557,2647,2738,2830,2922,3012,3103,3195,3287,3378,3469,3561,3653,3743,3834,3926,4018]
indices_end = [365,  456,  547,  639,  731,  821,  912, 1004,1096, 1186, 1277, 1369, 1461, 1551, 1642, 1734, 1826, 1917, 2009,2100, 2192, 2282, 2373, 2465, 2557, 2647, 2738, 2830, 2922, 3012,3103, 3195, 3287, 3378, 3469, 3561, 3653, 3743, 3834, 3926, 4018,4108,4199,4291,4383]

for indx in range(0,45):
    start = indices_start[indx]
    end = indices_end[indx]
    predictor_nc_u = np.asarray(DS_forecast.u10[start:end,:,:])
    predictor_nc_v = np.asarray(DS_forecast.v10[start:end,:,:])
    
    response_nc_u = np.asarray(DS_observation.u10[start:end,:,:])
    response_nc_v = np.asarray(DS_observation2.v10[start:end,:,:])
            #indxMasked = np.ma.getmaskarray(response)==False

    response_u = response_nc_u.flatten()
    response_v = response_nc_v.flatten()
    predictor_u = predictor_nc_u.flatten()
    predictor_v = predictor_nc_v.flatten()
    
    response = np.sqrt(np.square(response_u)+np.square(response_v))
    predictor = np.sqrt(np.square(predictor_u)+np.square(predictor_v))
        
        
    resOrder = np.argsort(response)
    resSort = response[resOrder] 
    preSort = predictor[resOrder]                
    preRank = rankdata(preSort, method='average')
    resRank = rankdata(resSort, method='average')
    resClass = rankdata(resSort, method='dense')
    CPA_All[m] = (np.cov(resClass,preRank)[0][1]/np.cov(resClass,resRank)[0][1]+1)/2 
    print(CPA_All[m])
    m = m+1
print(CPA_All)

In [ ]:
# compute summation in Precipitation 
# read in hourly data from ERA 5 by year. Note time coordinate is shifted by -1 hour to 
# correctly compute daily sum of precipitation (cdo shifttime,-1hour)
# sum dayofmonth --> first entry of every year need to added to last day of the previous year

target_obs = '/home/eva-maria/lsdf_data/Precipitation/Reanalysis/tp_sel_shift_2007.nc'
DS_observation = xr.open_dataset(target_obs)

target_obs2 = '/home/eva-maria/lsdf_data/Precipitation/Reanalysis/tp_sel_shift_2008.nc'
DS_observation2 = xr.open_dataset(target_obs2)

time = pd.date_range('2007-01-01', freq='24H', periods=365)

DS_2007 = DS_observation.tp[1:,:,:].groupby('time.dayofyear').sum('time')
DS_2007[364,:,:] = DS_2007[364,:,:] + DS_observation2.tp[0,:,:]
DS_2007 =DS_2007.assign_coords(dayofyear=(time))
DS_2007 = DS_2007.rename({'dayofyear':'time'})

# save file to disk
DS_2007.to_netcdf('/home/eva-maria/lsdf_data/Precipitation/Reanalysis/sum_tp_2007.nc')

# year 2008: leap year
target_obs3 = '/home/eva-maria/lsdf_data/Precipitation/Reanalysis/tp_sel_shift_2009.nc'
DS_observation3 = xr.open_dataset(target_obs3)

time = pd.date_range('2008-01-01', freq='24H', periods=366)

DS_2008 =  DS_observation2.tp[1:,:,:].groupby('time.dayofyear').sum('time')
DS_2008[365,:,:] = DS_2008[365,:,:] + DS_observation3.tp[0,:,:]
DS_2008 =DS_2008.assign_coords(dayofyear=(time))
DS_2008 = DS_2008.rename({'dayofyear':'time'})

# save file to disk
DS_2008.to_netcdf('/home/eva-maria/lsdf_data/Precipitation/Reanalysis/sum_tp_2008.nc')